Best practices:
1) Preprocessing and cleaning
2) Train Test Split
3)BOW,TFIDF, Word2Vec
4)Train ml algorithms

In [1]:
import pandas as pd
data = pd.read_csv("all_kindle_review.csv")


In [2]:
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
data = data[["reviewText", "rating"]]

In [4]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [5]:
data.shape

(12000, 2)

In [6]:
data.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [7]:
data['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [8]:
data['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [9]:
#####  preprocessing and cleaning


In [10]:
# Positive review is 1 negative reveiews are 0

data['rating']= data['rating'].apply(lambda x:0 if x<3 else 1)

In [11]:
data['rating']

0        1
1        1
2        1
3        1
4        1
        ..
11995    1
11996    1
11997    1
11998    0
11999    1
Name: rating, Length: 12000, dtype: int64

In [12]:
data.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",1
1,Great short read. I didn't want to put it dow...,1
2,I'll start by saying this is the first of four...,1
3,Aggie is Angela Lansbury who carries pocketboo...,1
4,I did not expect this type of book to be in li...,1


In [13]:
data['rating'].unique()

array([1, 0], dtype=int64)

In [14]:
### lower all the cases

data['reviewText'] = data['reviewText'].str.lower()

In [15]:
data.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [16]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
!pip install beautifulsoup4

In [18]:
!pip install lxml



In [19]:
from bs4 import BeautifulSoup

In [20]:
## Removing special characters
data['reviewText']=data['reviewText'].apply(lambda x:re.sub('[^a-z A-z 0-9-]+', '',x))
## Remove the stopswords
data['reviewText']=data['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## Remove url 
data['reviewText']=data['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
data['reviewText']=data['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
data['reviewText']=data['reviewText'].apply(lambda x: " ".join(x.split()))

In [21]:
data.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [22]:
### Lemmatizer

from nltk.stem import WordNetLemmatizer

In [23]:
lemmatizer = WordNetLemmatizer()

In [24]:
def lemmatize_word(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [25]:
data['reviewText'] = data['reviewText'].apply(lambda x:lemmatize_word(x))

In [26]:
data.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [27]:
## train test split

from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(data['reviewText'], data['rating'], test_size=0.20)

In [30]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()
xtrain_bow = bow.fit_transform(xtrain).toarray()
xtest_bow = bow.transform(xtest).toarray()

In [31]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
xtrain_tfidf = tfidf.fit_transform(xtrain).toarray()
xtest_tfidf = tfidf.transform(xtest).toarray()

In [32]:
from sklearn.naive_bayes import GaussianNB
nb_model_bow = GaussianNB().fit(xtrain_bow, ytrain)
nb_model_tfidf = GaussianNB().fit(xtrain_tfidf, ytrain)

In [33]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [34]:
ypred_bow = nb_model_bow.predict(xtest_bow)
ypred_tfidf = nb_model_tfidf.predict(xtest_tfidf)

In [35]:
print("BOW accuracy: ", accuracy_score(ytest, ypred_bow))

BOW accuracy:  0.5766666666666667


In [36]:
print("TFIDF accuracy: ", accuracy_score(ytest, ypred_tfidf))

TFIDF accuracy:  0.58


In [37]:
confusion_matrix(ytest,ypred_tfidf)


array([[481, 313],
       [695, 911]], dtype=int64)

In [38]:
confusion_matrix(ytest,ypred_bow)


array([[499, 295],
       [721, 885]], dtype=int64)